In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint

import datetime as dt

from scipy.stats import norm
from scipy.optimize import brentq
from scipy.optimize import least_squares

from sklearn.linear_model import LinearRegression

from scipy.integrate import quad

In [5]:
def h(x):
    return x ** (1/3) + (3/2) * np.log(x) +10.0
def h_prime(x):
    return (1/3) * (x**(-2/3)) + (3/2) * (1/x)
def h_prime_prime(x):
    return (-2/9) * (x**(-5/3)) + (3/2) * (-1/(x**2))

In [6]:
spx = pd.read_csv('SPX_options.csv')
spy = pd.read_csv('SPY_options.csv')
rate = pd.read_csv('zero_rates_20201201.csv')

In [7]:
rate['rate'] = rate['rate'] / 100

In [8]:
spx['mid'] = 0.5*(spx['best_bid'] + spx['best_offer'])
spx['strike'] = spx['strike_price'] / 1000
spx['payoff'] = spx['cp_flag'].map(lambda x: 'call' if x == 'C' else 'put')
spx.dropna(inplace=True)

r45 = np.interp(45,rate['days'],rate['rate'])

exdate = sorted(spx['exdate'].unique())[1]

df_x = spx[spx['exdate'] == exdate]

days_to_expiry = (pd.Timestamp(str(exdate)) - pd.Timestamp('2020-12-01')).days
T = days_to_expiry/365




In [9]:
spy['mid'] = 0.5*(spy['best_bid'] + spy['best_offer'])
spy['strike'] = spy['strike_price'] / 1000
spy['payoff'] = spy['cp_flag'].map(lambda x: 'call' if x == 'C' else 'put')
spy.dropna(inplace=True)

df_y = spy[spy['exdate'] == exdate]

In [10]:
df_x_call= df_x[df_x['cp_flag']=='C']
df_x_put = df_x[df_x['cp_flag']=='P']

S_x = 3662.45
K_x = 3662.45
F_x = S_x* np.exp(r45*T)

call_mid_x = np.interp(K_x,df_x_call['strike'],df_x_call['mid'])
put_mid_x = np.interp(K_x,df_x_put['strike'],df_x_put['mid'])

In [11]:
df_y_call= df_y[df_y['cp_flag']=='C']
df_y_put = df_y[df_y['cp_flag']=='P']

S_y = 366.02
K_y = 366.02
F_y = S_y* np.exp(r45*T)

call_mid_y = np.interp(K_y,df_y_call['strike'],df_y_call['mid'])
put_mid_y = np.interp(K_y,df_y_put['strike'],df_y_put['mid'])

# BSM

In [12]:
def BlackScholesCall(S,K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

def BlackScholesPut(S, K, r, sigma, T):
    return BlackScholesCall(S, K, r, sigma, T) - S + K*np.exp(-r*T)

def callintegrandBSM(K, S, r, T, sigma):
    price = BlackScholesCall(S, K, r, sigma, T) / K**2
    return price

def putintegrandBSM(K, S, r, T, sigma):
    price = BlackScholesPut(S, K, r, sigma, T) / K**2
    return price

def impliedVolatilityBSM(S, K, r, price, T, payoff):
    try:
        if (payoff.lower() == 'call'):
            impliedVol = brentq(lambda x: price -
                                BlackScholesCall(S, K, r, x, T),
                                1e-12, 10.0)
        elif (payoff.lower() == 'put'):
            impliedVol = brentq(lambda x: price -
                                BlackScholesPut(S, K, r, x, T),
                                1e-12, 10.0)
        else:
            raise NameError('Payoff type not recognized')
    except Exception:
        impliedVol = np.nan

    return impliedVol

>spx

In [13]:
call_sigma_BSM_x = impliedVolatilityBSM(S_x,K_x,r45,call_mid_x,T,payoff = 'call')
put_sigma_BSM_x = impliedVolatilityBSM(S_x,K_x,r45,put_mid_x,T,payoff = 'put')
sigma1_x = (call_sigma_BSM_x+put_sigma_BSM_x)/2

In [14]:
I_put_BSM_x = quad(lambda x: putintegrandBSM(x, S_x, r45, T, sigma1_x), 0.0, F_x)
I_call_BSM_x = quad(lambda x: callintegrandBSM(x, S_x, r45, T, sigma1_x), F_x, 5000)
E_var_BSM_x = 2*np.exp(r45*T)*(I_put_BSM_x[0] + I_call_BSM_x[0])
print('The expected integrated variance is: %.9f' % E_var_BSM_x)

The expected integrated variance is: 0.004213733


In [15]:
sigma_BSM_x = np.sqrt(E_var_BSM_x/T)
sigma_BSM_x

0.18487308494097424

In [16]:
def get_price_BSM(S,r,T,sigma):
    return np.exp(-r*T)*\
            (S**(1/3) * np.exp((1/3)*r - (1/9)*(sigma**2)*T)\
            +(3/2)*(np.log(S) + (r-sigma**2/2)*T)
            +10)

In [17]:
get_price_BSM(S_x,r45,T,sigma1_x)

37.71413781849898

>spy

In [18]:
call_sigma_BSM_y = impliedVolatilityBSM(S_y,K_y,r45,call_mid_y,T,payoff = 'call')
put_sigma_BSM_y = impliedVolatilityBSM(S_y,K_y,r45,put_mid_y,T,payoff = 'put')
sigma1_y = (call_sigma_BSM_y+put_sigma_BSM_y)/2

In [19]:
I_put_BSM_y = quad(lambda x: putintegrandBSM(x, S_y, r45, T, sigma1_y), 0.0, F_y)
I_call_BSM_y = quad(lambda x: callintegrandBSM(x, S_y, r45, T, sigma1_y), F_y, 500)
E_var_BSM_y = 2*np.exp(r45*T)*(I_put_BSM_y[0] + I_call_BSM_y[0])
print('The expected integrated variance is: %.9f' % E_var_BSM_y)

The expected integrated variance is: 0.004209706


In [20]:
sigma_BSM_y = np.sqrt(E_var_BSM_y/T)
sigma_BSM_y

0.18478472009599142

In [21]:
get_price_BSM(S_y,r45,T,sigma1_y)

25.999443889743038

# Bachelier

In [22]:
def Bachelier_vanilla_call(S, K, r, sigma, T) :
    d1 = (K - S) / (sigma * np.sqrt(T))
    Vc = np.exp(-r * T) * ((S - K) * norm.cdf(-d1) + sigma * np.sqrt(T) * norm.pdf(-d1))
    return Vc

def Bachelier_vanilla_put(S, K, r, sigma, T) :
    d1 = (K - S) / (sigma * np.sqrt(T))
    Vp = np.exp(-r * T) * ((K - S) * norm.cdf(d1) + sigma * np.sqrt(T) * norm.pdf(d1))
    return Vp

def callintegrandBachelier(K, S, r, T, sigma):
    price = Bachelier_vanilla_call(S, K, r, sigma, T) / K**2
    return price

def putintegrandBachelier(K, S, r, T, sigma):
    price = Bachelier_vanilla_put(S, K, r, sigma, T) / K**2
    return price

def impliedVolatilityBachelier(S, K, r, price, T, payoff):
    try:
        if (payoff.lower() == 'call'):
            impliedVol = brentq(lambda x: price -
                                Bachelier_vanilla_call(S, K, r, x, T),
                                1e-12, 1000)
        elif (payoff.lower() == 'put'):
            impliedVol = brentq(lambda x: price -
                                Bachelier_vanilla_put(S, K, r, x, T),
                                1e-12, 1000)
        else:
            raise NameError('Payoff type not recognized')
    except Exception:
        impliedVol = np.nan

    return impliedVol

>spx

In [23]:
call_sigma_Bachelier_x = impliedVolatilityBachelier(S_x,K_x,r45,call_mid_x,T,payoff = 'call')
put_sigma_Bachelier_x = impliedVolatilityBachelier(S_x,K_x,r45,put_mid_x,T,payoff = 'put')
sigma2_x = (call_sigma_Bachelier_x+put_sigma_Bachelier_x)/2

In [24]:
sigma2_x

677.0602010158082

In [25]:
I_put_BCH_x = quad(lambda x: putintegrandBachelier(x, S_x, r45, T, sigma2_x), 0.0, F_x)
I_call_BCH_x = quad(lambda x: callintegrandBachelier(x, S_x, r45, T, sigma2_x), F_x, 5000)
E_var_BCH_x = 2*np.exp(r45*T)*(I_put_BCH_x[0] + I_call_BCH_x[0])
print('The expected integrated variance is: %.9f' % E_var_BCH_x)

The expected integrated variance is: 0.004240457


In [26]:
sigma_BCH_x = np.sqrt(E_var_BCH_x/T)
sigma_BCH_x

0.1854584018888951

In [27]:
def static_replication_BCH(F,S, r, T, sigma,upperbound):
    ans =  (quad(lambda x:
                h_prime_prime(x)*Bachelier_vanilla_put(S,x, r, sigma, T),0.0,F
                )[0]
            +
            quad(lambda x:
                h_prime_prime(x)*Bachelier_vanilla_call(S,x, r, sigma, T), F , upperbound
                )[0]
            )
    return np.exp(-r45*T)*h(F) + ans

In [28]:
static_replication_BCH(F_x,S_x,r45, T ,sigma2_x,5000)

37.70484676740247

>spy

In [29]:
call_sigma_Bachelier_y = impliedVolatilityBachelier(S_y,K_y,r45,call_mid_y,T,payoff = 'call')
put_sigma_Bachelier_y = impliedVolatilityBachelier(S_y,K_y,r45,put_mid_y,T,payoff = 'put')
sigma2_y = (call_sigma_Bachelier_y+put_sigma_Bachelier_y)/2

In [30]:
sigma2_y

67.63193548527376

In [31]:
I_put_BCH_y = quad(lambda x: putintegrandBachelier(x, S_y, r45, T, sigma2_y), 0.0, F_y)
I_call_BCH_y = quad(lambda x: callintegrandBachelier(x, S_y, r45, T, sigma2_y), F_y, 500)
E_var_BCH_y = 2*np.exp(r45*T)*(I_put_BCH_y[0] + I_call_BCH_y[0])
print('The expected integrated variance is: %.9f' % E_var_BCH_y)

The expected integrated variance is: 0.004236360


In [32]:
sigma_BCH_y = np.sqrt(E_var_BCH_y/T)
sigma_BCH_y

0.18536877806126525

In [33]:
static_replication_BCH(F_y,S_y,r45, T ,sigma2_y,500)

25.995121607184974

# Static-replication

In [34]:
def SABR(F, K, T, alpha, beta, rho, nu):
    beta = 0.7
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma


In [35]:
def SABRCall(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
    return BlackScholesCall(S, K, r, sabr_vol, T)


def SABRPut(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
    return BlackScholesPut(S, K, r, sabr_vol, T)


def sabrcallintegrand(K, S, r, T, alpha, beta, rho, nu):
    price = SABRCall(S, K, r, alpha, beta, rho, nu, T) / K**2
    return price


def sabrputintegrand(K, S, r, T, alpha, beta, rho, nu):
    price = SABRPut(S, K, r, alpha, beta, rho, nu, T) / K**2
    return price

In [36]:
alpha1 = 1.817
beta1 = 0.7
rho1 = -0.404
nu1 = 2.790

In [37]:
I_put_SR_x = quad(lambda x: sabrputintegrand(x, S_x, r45, T, alpha1, beta1, rho1, nu1), 1e-6, F_x)
I_call_SR_x = quad(lambda x: sabrcallintegrand(x, S_x, r45, T, alpha1, beta1, rho1, nu1), F_x, 5000)
E_var_SABR_x = 2*np.exp(r45*T)*(I_put_SR_x[0] + I_call_SR_x[0])
print('The expected integrated variance is: %.9f' % E_var_SABR_x)

The expected integrated variance is: 0.006337324


In [38]:
sigma_SABR_x = np.sqrt(E_var_SABR_x/T)
sigma_SABR_x

0.2267217135315304

In [39]:
def static_replication_SABR(F, S, T, alpha, beta, rho, nu,r,upperbound):
    ans =  (quad(lambda x:
                h_prime_prime(x)*SABRPut(S, x, r, alpha, beta, rho, nu, T),0.0,F
                )[0]
            +
            quad(lambda x:
                h_prime_prime(x)*SABRCall(S, x, r, alpha, beta, rho, nu, T), F , upperbound
                )[0]
            )
    return np.exp(-r45*T)*h(F) + ans

In [40]:
static_replication_SABR(F_x, S_x, T, alpha1, beta1, rho1, nu1,r45,5000)

37.70040702356144

>spy

In [41]:
alpha2 = 0.908
beta2 = 0.7 
rho2 = -0.489
nu2 = 2.729

In [42]:
I_put_SR_y = quad(lambda x: sabrputintegrand(x, S_y, r45, T, alpha2, beta2, rho2, nu2), 1e-6, F_y)
I_call_SR_y = quad(lambda x: sabrcallintegrand(x, S_y, r45, T, alpha2, beta2, rho2, nu2), F_y, 5000)
E_var_SABR_y = 2*np.exp(r45*T)*(I_put_SR_y[0] + I_call_SR_y[0])
print('The expected integrated variance is: %.9f' % E_var_SABR_y)

The expected integrated variance is: 0.006016732


In [43]:
sigma_SABR_y = np.sqrt(E_var_SABR_y/T)
sigma_SABR_y

0.2209126041673096

In [44]:
static_replication_SABR(F_y, S_y, T, alpha2, beta2, rho2, nu2,r45,500)

25.99267260991541

In [54]:
df = pd.DataFrame(index=[['SPX','SPY']],
                  columns= pd.MultiIndex.from_tuples([('BSM', 'sigma'), 
                                                      ('BSM', 'pricing'), 
                                                      ('BACH', 'sigma'), 
                                                      ('BACH', 'pricing'),
                                                      ('Sta-Repl','sigma'),
                                                      ('Sta-Repl','pricing')],
                                    )
)

In [58]:
df.iloc[:,0]=[0.004213733,0.004209706]
df.iloc[:,1]=[37.71413781849898,25.999443889743038]
df.iloc[:,2]=[0.004240457,0.004236360]
df.iloc[:,3]=[37.70484676740247,25.995121607184974]
df.iloc[:,4]=[0.006337324,0.006016732]
df.iloc[:,5]=[37.70040702356144,25.99267260991541]


In [59]:
df

BSM                 BACH             Sta-Repl           
        sigma    pricing     sigma    pricing     sigma    pricing
SPX  0.004214  37.714138   0.00424  37.704847  0.006337  37.700407
SPY   0.00421  25.999444  0.004236  25.995122  0.006017  25.992673